積極的に小さい関数を作り、テストをしやすくする。

In [6]:
import numpy as np
import math


def TIP4PSites():
    """
    TIP4Pの座標系
    """

    angle,holen,colen = 104.52,0.9572,0.15
    wh = 1.0
    wo = 16.0
    wm = wh + wh + wo
    rangle = angle*math.pi/360
    ohz=holen*math.cos(rangle)
    hyl=holen*math.sin(rangle)
    hzl=wo*ohz/wm
    ol=-ohz+hzl
    cl=colen+ol

    # 分子内の原子位置
    return np.array([[0,hyl,hzl],
                      [0,-hyl,hzl],
                      [0,0,ol],
                      [0,0,cl]]).T, np.array([wh,wh,wo,0.0])


In [7]:
# 分子の座標を読みこむ。
from math import sin,cos,pi,acos

def quat2euler(q):
    e = np.zeros(3)
    if q[0] == 1.0:
        return e
    p = 2 * (q[0]**2 + q[3]**2) - 1
    if p > 1.0:
        p = 1.0
    if p < -1.0:
        p = -1.0
    e[0] = acos(p)
    thh = e[0] / 2.0
    sinthh = sin(thh)
    costhh = cos(thh)
    p = q[0] / costhh
    if p > 1.0:
        p = 1.0
    if p < -1.0:
        p = -1.0
    p = acos(p)
    if sinthh == 0.0:
        s = 1.0
    else:
        s = q[1] / sinthh
    if s > 1.0:
        s = 1.0
    if s < -1.0:
        s = -1.0
    s = acos(s)
    if q[3] < 0.0:
        p = 2 * pi - p
    if q[2] > 0:
        e[2] = p + s
        e[1] = p - s
    else:
        e[2] = p - s
        e[1] = p + s
    e[1:3] %= (2 * pi)
    return e

def LoadNX3A(file):
    # file = open("9.nx3a")
    while True:
        line = file.readline()
        if len(line) == 0:
            break
        if "@NX3A" in line:
            line = file.readline()
            nmol = int(line)
            pos = []
            for i in range(nmol):
                line = file.readline()
                pos.append([float(x) for x in line.split()])
            pos = np.array(pos)
        if "@NX4A" in line:
            line = file.readline()
            nmol = int(line)
            pos = []
            for i in range(nmol):
                line = file.readline()
                x,y,z,a,b,c,d = [float(x) for x in line.split()]
                ea,eb,ec = quat2euler([a,b,c,d])
                pos.append([x,y,z,ea,eb,ec])
            pos = np.array(pos)
        if "@BOX3" in line:
            line = file.readline()
            cell = np.diag(np.array([float(x) for x in line.split()]))
    return cell, pos



In [8]:
#import importlib
#importlib.reload(Symbolic)
from Symbolic import phic, phic_r, phic_rr, phiLJ, phiLJ_r, phiLJ_rr, Trunc, Trunc_r, Trunc_rr, A, Aa, Ab, Ac, Aaa, Aab, Aca, Abb, Abc, Acc, KETensor

Object `Trunc_rr` not found.
Object `KETensor` not found.


In [9]:
# 相互作用と分子形状に関するブロック


# 当面、田中プログラムとの照合のために、エネルギーの単位は、
# 水素2原子が1 Aにいる時のCoulomb力を1とする。

import math
# SI unit 2019
Na=6.02214076e23    #Avogadro
ee=1.60217662e-19   #Elementary charge
E0=8.8541878128e-12 #Vacuum permittivity
UJ = 4.184          #1 cal in J
qe0 = ee**2/(4*math.pi*E0)*Na/UJ*1e7
qeT = 332.17752e0
sw = (qeT*UJ*1e6/18.0)**0.5/(6.0*math.pi)
# Tanaka's Units
TanakaScale = qeT / qe0
TokJmol = ee**2/(4*math.pi*E0*1e-7)*Na*TanakaScale # 内部単位をkJ/molに変換する。
print("TanakaScale", TanakaScale)


charge = [0.52,0.52,0,-1.04]
LJpairs = {(2, 2):(6.0e5*UJ / TokJmol, 6.10e2*UJ / TokJmol)}
CCpairs = dict()
for i,ci in enumerate(charge):
    for j,cj in enumerate(charge):
        cc = ci*cj
        if cc:
            CCpairs[i,j] = cc
            


TanakaScale 1.0003427429781058


In [10]:
# 相互作用をサイト間距離で微分したもの。
# この辞書がNUMBAとフィットしない。もうすこし方法はないものか。
# 相対位置ベクトルと、関数の微分値を別の配列にしてしまう、か。

def derivatives(CCpairs, LJpairs, v, D0i, D0j):
    imax = 0
    jmax = 0
    for i, j in CCpairs:
        imax = max(i, imax)
        jmax = max(j, jmax)
    imax += 1
    jmax += 1
    ints = np.zeros([imax, jmax], dtype=bool)
    phis = np.zeros([imax, jmax, 3])
    seps = np.zeros([imax, jmax, 3])
    
    for (i,j),cc in CCpairs.items():
        ints[i,j] = True
        r         = D0i[:,i] - D0j[:,j] + v
        seps[i,j] = r
        rL = np.linalg.norm(r)
        phi    = cc*phic(rL)
        phi_r  = cc*phic_r(rL)
        phi_rr = cc*phic_rr(rL)
        phis[i,j] += np.array([phi, phi_r, phi_rr])

    for (i,j),(a1,a2) in LJpairs.items():
        ints[i,j] = True
        r         = D0i[:,i] - D0j[:,j] + v
        seps[i,j] = r
        rL = np.linalg.norm(r)
        phi    = phiLJ(rL,a1,a2)
        phi_r  = phiLJ_r(rL,a1,a2)
        phi_rr = phiLJ_rr(rL,a1,a2)
        phis[i,j] += np.array([phi, phi_r, phi_rr])

    return ints,seps,phis


In [46]:
def E_pair(ints, phis, ramp):
    """
    
    Args:
        ints (2nd rank numpy array of boolean): Pairs of interaction.
        phis (3rd rank numpy array of float:    Interaction and its derivatives
        ramp (float) : Ramp function
    """
    # print(ramp*phis[2,2,0]*TokJmol) # 2,2 == Oxygen to Oxygen, 0th is interaction
    return ramp * np.sum(phis[ints, 0]) # 0th is interaction
    # return ramp * np.sum(phis[:,:,0]) # 0th is interaction

In [12]:
intra, mass = TIP4PSites()

# for a rigid rotor
IM = np.array([mass @ np.sum(intra[1:3]**2, axis=0),
               mass @ np.sum(intra[0:3:2]**2, axis=0),
               mass @ np.sum(intra[0:2]**2, axis=0)])
print(IM)

# 各サイトの、各軸からの変位の二乗
squareoffsets = np.sum(intra**2,axis=0) - (intra**2)

IM = squareoffsets @ mass
print(IM)

[1.75618412 0.61023652 1.1459476 ]
[1.75618412 0.61023652 1.1459476 ]


In [47]:
file = open("test.nx3a")
file = open("tanaka20191215.nx3a")

cell, pos = LoadNX3A(file)

# Tanaka programでの配列要素(1,2)は、Pythonでは[0,1]になる。
# 以後、縦横を精査。
cell

array([[27.2491,  0.    ,  0.    ],
       [ 0.    , 27.2491,  0.    ],
       [ 0.    ,  0.    , 27.4689]])

In [48]:
# あらかじめ行列を準備しておく。これをアップデートする。
nmol = pos.shape[0]
R = np.zeros([nmol,3,3])
D0 = np.zeros([nmol,3,4])

for im in range(nmol):
    ea, eb, ec = pos[im, 3:6]
    R[im] = A(ea,eb,ec)
    D0[im] = R[im] @ intra

## Quenching

まず力を算出する。

### 変位の与え方

Overdamped motionとは、慣性的な動きが摩擦によっていつも打ち消され、常に力の方向に力に比例して進む状態と考えればいい。
加速度は(力/質量)に比例する。速度は加速度の分だけ増えるが、もともと速度が0であれば、速度もまた加速度に比例する。
力に比例して質点を動かすと、自然にエネルギーの極小に落ちこむ。

### 並進運動

$$\mathbf{F} = -{\partial \phi\over \partial \mathbf{r}}$$
この力により、通常の運動方程式であれば、分子は加速度を得るが、Quenchの場合はそれを速度に読みかえる。
${\mathbf{F}\over m}$に比例して並進させる。

### 回転運動

Overdamped motionでは速度も角速度もいつも0とみなして構わない。そうすると、剛体軸周りの角加速度は、剛体軸周りのトルクを、その軸周りの慣性モーメントで割って得られる。(Goldstein 5-39')

トルクはベクトルで表される。

1. 質点の相互作用からトルクを計算
2. それを剛体座標に変換
3. 角加速度を求める
4. 回転行列の微小変化を求める
5. そして新しい配置でまた相互作用を計算する。

こんな手順か。

In [39]:
def ForceAndTorque(pos, cell, D0, Rin, Transition):
    nmol = pos.shape[0]
    celli = np.linalg.inv(cell)
    force = np.zeros([nmol,3])
    torque = np.zeros([nmol,3])
    ep = 0.0
    for im in range(nmol):
        for jm in range(im+1,nmol):
            # 重心間相対ベクトル
            v = pos[im][:3] - pos[jm][:3]
            # 周期境界条件
            v -= cell @ np.floor( celli @ v + 0.5 )
            # 重心間距離
            vL = np.linalg.norm(v)
            print("vL",vL)

            # 切断関数を準備。
            trunc = False
            if vL > Rin + Transition:
                # 遠すぎる対はパス
                continue
            if vL > Rin:
                # カットオフ関数がかかる場合
                ramp = Trunc(vL, Rin, Transition)
                ramp_r = Trunc_r(vL, Rin, Transition)
            else:
                # 近距離
                ramp = 1.0
                ramp_r = 0.0
            # print(D0im.shape)
            ints, seps, phis = derivatives(CCpairs, LJpairs, v, D0[im], D0[jm])

            ep += E_pair(ints, phis, ramp)

            for si,sj in zip(*np.where(ints)):
                r = seps[si,sj]
                phi, phi_r, phi_rr = phis[si,sj]
                rL = np.linalg.norm(r)
                F = -(phi_r*ramp * r / rL + phi*ramp_r*v/vL)
                torque[im] += np.cross(D0[im,:,si], F)
                torque[jm] += np.cross(D0[jm,:,sj], -F)
                force[im] +=  F
                force[jm] += -F

    return force, torque, ep


両方同時に実施する。



In [40]:
def Quench(pos, cell, D0, unfixed=None):
    wm = np.sum(mass)

    lastep = 99
    nmol = pos.shape[0]
    count = 1
    nextshow = 1

    while True:
        force, torque, ep = ForceAndTorque(pos, cell, D0, Rin=6.5, Transition=2.0)
        if count == nextshow:
            print("{0} energy (kJ/mol)={1}".format(count,ep*TokJmol/nmol))
            # check orthogonality and unitality
            print(R[0,:,0] @ R[0,:,1], R[0,:,1] @ R[0,:,2], R[0,:,2] @ R[0,:,0], np.linalg.det(R))
            # あまり変形が大きいようなら特異値分解で修正する。
            nextshow = nextshow * 2


        # 本来、力を質量で割ったものは加速度だが、それを速度に読みかえる。
        v = (force/wm)*1.0
        if unfixed is None:
            pos[:,:3] += v*5.0
        else:
            for im in range(nmol):
                if unfixed[im]:
                    pos[im,:3] += v[im]*5.0

        for im in range(nmol):
            if unfixed is not None and not unfixed[im]:
                continue
            # torqueはベクトルで表現される。
            # 空間座標でのtorqueを剛体座標になおす。
            tm = R[im].T @ torque[im]
            # 本来、Torqueを慣性モーメントで割ったものは角加速度だが、それを角速度に読みかえる。
            omega = tm / IM
            # 再び空間座標に戻す。
            om = (R[im] @ omega) * 5.0
            # 微小回転行列
            dR = np.array([[ 1,-om[2], om[1]],
                           [ om[2], 1,-om[0]],
                           [-om[1], om[0], 1]])
            # を掛ける
            R[im] = dR @ R[im]
            # 原子位置も再計算
            D0[im] = R[im] @ intra

        assert ep < lastep
        ee = abs((lastep - ep) / ep)
        if ee < 1e-8:
            return ep, pos, count
        count += 1
        lastep = ep

In [49]:
ep, qpos, count = Quench(pos, cell, D0)
print("{0} energy (kJ/mol)={1}".format(count,ep*TokJmol/nmol))

vL 2.84674109816822
4.067227282648917
1 energy (kJ/mol)=-9.849064744824588
0.0 0.0 0.0 [1. 1.]
vL 2.843570452201347
4.187141355504938
2 energy (kJ/mol)=-10.259279465201708
1.3851039713769985e-06 2.233905001585296e-06 -1.2174161262423766e-09 [1.00254161 1.00076712]
vL 2.8403985236445437
4.310746709315792
vL 2.837316982736551
4.434735372364741
4 energy (kJ/mol)=-10.783220654205833
3.0431076617223373e-06 4.867815068888907e-06 -2.342833371749009e-09 [1.00643884 1.00148128]
vL 2.8343737174970207
4.556999889150782
vL 2.8315909433584805
4.67621869976966
vL 2.82897618702693
4.7915777722145085
vL 2.826528756790659
4.902590761277989
8 energy (kJ/mol)=-11.324332760104907
4.886307762785153e-06 7.545873007242041e-06 -3.3076395511422163e-09 [1.01156926 1.00177871]
vL 2.8242435060049296
5.008984409673763
vL 2.8221130072487166
5.110625636371545
vL 2.820128799042695
5.207474929083183
vL 2.818282091568223
5.299556300132729
vL 2.816564156732403
5.38693769542604
vL 2.814966534536196
5.469718023310717
vL 2

In [29]:
celli = np.linalg.inv(cell)
for im in range(nmol):
    for jm in range(im+1,nmol):
        # 重心間相対ベクトル
        v = pos[im][:3] - pos[jm][:3]
        # 周期境界条件
        v -= cell @ np.floor( celli @ v + 0.5 )
        # 重心間距離
        vL = np.linalg.norm(v)
        print(im,jm,vL)


        ints, seps, phis = derivatives(CCpairs, LJpairs, v, D0[im], D0[jm])

        for si,sj in zip(*np.where(ints)):
            r = seps[si,sj]
            rL = np.linalg.norm(r)
            print(im,jm,si,sj,rL)

0 1 2.84674109816822
0 1 0 0 2.410468664212192
0 1 0 1 3.5614719053848485
0 1 0 3 3.218832956697071
0 1 1 0 2.4111977436059044
0 1 1 1 3.5624730217633687
0 1 1 3 3.2196598020447715
0 1 2 2 2.84674109816822
0 1 3 0 1.98305094389639
0 1 3 1 3.2875556661366985
0 1 3 3 2.84674109816822


Quenchはこれで完成。多成分への拡張など必要になるが、これに関しては自分でこれ以上書くよりも、Gromacsにまかせるのが賢明だと思う。

# 局所Quench

指定された分子と、それに隣接する4分子だけを選び、そのうち隣接分子は移動を許さず、指定された分子のみの最適配置を計算する。上のプロシージャをすこし加工するだけでいい、はず。

In [554]:
# 読み込む。注目する分子を指定
file1 = "ices/6.q.nx4a"
file2 = "ices/6x.q.nx4a"
file = open(file1)
cell, pos = LoadNX3A(file)

target = 4

In [555]:
# 注目する分子とその隣接分子以外の情報を削除
import pairlist as pl

rpos = pos[:,:3] @ np.linalg.inv(cell).T
# print(rpos)
nei = [target,]
for i,j,d in pl.pairs_iter(rpos, 3.05, cell):
    if i == target:
        nei.append(j)
    elif j == target:
        nei.append(i)
print(nei)
pos = pos[nei]
pos

[4, 32, 167, 3, 9]


array([[ 1.68156076,  2.93123222,  4.88526457,  1.29541721,  2.42531758,
         5.96556183],
       [24.56373817,  4.60943655,  3.60805945,  1.96486169,  0.90351062,
         3.5361969 ],
       [ 1.70435753,  4.57803529,  7.07969239,  1.5889283 ,  3.9270565 ,
         5.62619219],
       [ 3.32319787,  4.62753466,  3.47358363,  0.96901766,  2.46453154,
         1.94440751],
       [ 1.61342912,  0.15560395,  5.01273612,  2.19892848,  2.3101165 ,
         1.99363954]])

In [556]:
# あらかじめ行列を準備しておく。これをアップデートする。
nmol = pos.shape[0]
R = np.zeros([nmol,3,3])
D0 = np.zeros([nmol,3,4])

for im in range(nmol):
    ea, eb, ec = pos[im, 3:6]
    R[im] = A(ea,eb,ec)
    D0[im] = R[im] @ intra

In [557]:
# 注目する分子以外は固定してQuench, 相対位置を求めておく。
unfixed = [True, False, False, False, False]
ep, qpos = Quench(pos, cell, D0, unfixed)
rqpos = qpos[:,:3] @ np.linalg.inv(cell).T

1 energy (kJ/mol)=-18.76206864565277
-0.0002548604989348613 0.0005875224564507175 0.0002507578735070348 [1.00128587 1.         1.         1.         1.        ]
2 energy (kJ/mol)=-18.773237755324654
-0.0002611599719514468 0.0006009056456340867 0.00025926388223300456 [1.00131785 1.         1.         1.         1.        ]
4 energy (kJ/mol)=-18.780233584546018
-0.0002626256065140775 0.0006038694503212039 0.0002620400841594761 [1.00132644 1.         1.         1.         1.        ]
8 energy (kJ/mol)=-18.788428515983334
-0.00026266053004314616 0.0006039359092562813 0.0002626230761755277 [1.00132797 1.         1.         1.         1.        ]
16 energy (kJ/mol)=-18.798261442138735
-0.00026259903347553015 0.000603781927337177 0.0002629882321835697 [1.00132905 1.         1.         1.         1.        ]
32 energy (kJ/mol)=-18.805960309029683
-0.00026254355894625436 0.0006036370989913609 0.0002632726883088493 [1.00132993 1.         1.         1.         1.        ]
64 energy (kJ/mol)=-18.8

In [558]:
# 単純に1.01倍する。
# 本当は、隣接分子の位置は変化するはずだが、局所の、結合の長さの違いがもっとも重要なのなら、
# 隣接分子の細かい移動など気にしなくても、変位の方向は割りだせるはず。
xpos  = qpos.copy()
xpos[:,:3] *= 1.01
xcell = cell * 1.01
ep, qxpos = Quench(xpos, xcell, D0, unfixed)
rqxpos = qxpos[:,:3] @ np.linalg.inv(xcell).T

1 energy (kJ/mol)=-18.70575755662876
-0.0002636467969311662 0.0006031876763138433 0.0002633637218766305 [1.00134578 1.         1.         1.         1.        ]
2 energy (kJ/mol)=-18.70602570908761
-0.00026368631594150815 0.0006032417837179005 0.0002633693739498566 [1.00134618 1.         1.         1.         1.        ]
4 energy (kJ/mol)=-18.70636931671463
-0.0002636958445867976 0.0006032705165083252 0.0002633729653130912 [1.00134627 1.         1.         1.         1.        ]
8 energy (kJ/mol)=-18.70682480468881
-0.0002636961872341803 0.0006032745683325902 0.0002633735306718926 [1.00134628 1.         1.         1.         1.        ]
16 energy (kJ/mol)=-18.70726641062644
-0.0002636961944087468 0.0006032742011253811 0.00026337341279629445 [1.00134629 1.         1.         1.         1.        ]
32 energy (kJ/mol)=-18.707487913039152
-0.00026369618953134277 0.0006032736095867297 0.00026337345657953537 [1.0013463 1.        1.        1.        1.       ]


In [559]:
# セル内での変位を求める。
Predict = (rqxpos[0]-rqpos[0]) @ cell / 0.01
Predict

array([ 0.18576598,  0.69273276, -0.26657003])

In [560]:
# 方位ベクトルの和による予測。
# Quenchすら行わない。こっちはかなり精度悪い。
dd = pos[1:,:3] - pos[0,:3]
for i in range(dd.shape[0]):
    dd[i] /= np.linalg.norm(dd[i])
print(dd)
Predict2 = np.sum(dd, axis=0)
Predict2

[[ 0.9958121   0.07380611 -0.05395294]
 [ 0.00343152  0.59749166  0.80186778]
 [ 0.59568724  0.62666022 -0.50244768]
 [-0.02942975 -0.99777011  0.05990574]]


array([1.56550112, 0.30018788, 0.3053729 ])

In [561]:
# 照合のため、全体をQuenchした場合の変位も求める。
# そこそこ一致するが、一致しないものもある(氷3の#5分子とか)
file = open(file1)
cell0, pos0 = LoadNX3A(file)
rpos0 = pos0[:,:3] @ np.linalg.inv(cell0).T
file = open(file2)
cell1, pos1 = LoadNX3A(file)
rpos1 = pos1[:,:3] @ np.linalg.inv(cell1).T
# 伸張によるセル内での相対座標の変位
drpos = rpos1 - rpos0
# 周囲4分子の変位をさしひく。
drpos[target] -= np.sum(drpos[nei[1:]], axis=0)/4.0
# 注目する分子の変位を絶対座標で表示
Actual = drpos[target] @ cell0 / 0.01
Actual





array([-0.164772  ,  0.27604774,  0.12468044])

In [562]:
Actual @ Predict / ( np.linalg.norm(Actual) * np.linalg.norm(Predict)),  np.linalg.norm(Predict)

In [553]:
 Actual @ Predict2 / ( np.linalg.norm(Actual) * np.linalg.norm(Predict2))

氷IIIの最初の単位胞の12分子のうち、#0#5#6#9は予測があわない。

これらは氷IIIをc軸から見た時の正方形に属しない分子と思われる。

斥力が効いているか、可能性はある。